# How to Build a Recommendation System for Purchase Data (Step-by-Step)
* Description: A documentation on building collaborative filtering models for recommending products to customers
* Link: https://medium.com/datadriveninvestor/how-to-build-a-recommendation-system-for-purchase-data-step-by-step-d6d7a78800b6
* Author: Moorissa Tjokro

## Problem statement
In this data challenge, we are building collaborative filtering models for recommending product items. The steps below aim to recommend users their top 10 items to place into their basket. The final output will be a csv file in the `output` folder, and a function that searches for a recommendation list based on a speficied user:
* Input: user - customer ID
* Returns: ranked list of items (product IDs), that the user is most likely to want to put in his/her (empty) "basket"

## 1. Import modules
* `pandas` and `numpy` for data manipulation
* `turicreate` for performing model selection and evaluation
* `sklearn` for splitting the data into train and test set

In [0]:
!pip install turicreate
!pip install sklearn
!pip install scripts

In [0]:
%load_ext autoreload
%autoreload 2

import pandas as pd
import numpy as np
import time
import turicreate as tc
from sklearn.model_selection import train_test_split

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [0]:
from google.colab import files
uploaded = files.upload()

Saving recommend_1.csv to recommend_1.csv
Saving trx_data.csv to trx_data.csv


## 2. Load data
Two datasets are used in this exercise, which can be found in `data` folder: 
* `recommend_1.csv` consisting of a list of 1000 customer IDs to recommend as output
* `trx_data.csv` consisting of user transactions

The format is as follows.

In [0]:
import io

customers = pd.read_csv(io.BytesIO(uploaded['recommend_1.csv'])) 
transactions = pd.read_csv(io.BytesIO(uploaded['trx_data.csv']))

In [0]:
print(customers.shape)
customers.head()

(1000, 1)


,customerId
0,1553
1,20400
2,19750
3,6334
4,27773


In [0]:
print(transactions.shape)
transactions.head()

(62483, 2)


,customerId,products
0,0,20
1,1,2|2|23|68|68|111|29|86|107|152
2,2,111|107|29|11|11|11|33|23
3,3,164|227
4,5,2|2


## 3. Data preparation
* Our goal here is to break down each list of items in the `products` column into rows and count the number of products bought by a user

In [0]:
# example 1: split product items
transactions['products'] = transactions['products'].apply(lambda x: [int(i) for i in x.split('|')])
transactions.head(2).set_index('customerId')['products'].apply(pd.Series).reset_index()

,customerId,0,1,2,3,4,5,6,7,8,9
0,0,20.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,1,2.0,2.0,23.0,68.0,68.0,111.0,29.0,86.0,107.0,152.0


In [0]:
# example 2: organize a given table into a dataframe with customerId, single productId, and purchase count
pd.melt(transactions.head(2).set_index('customerId')['products'].apply(pd.Series).reset_index(), 
             id_vars=['customerId'],
             value_name='products') \
    .dropna().drop(['variable'], axis=1) \
    .groupby(['customerId', 'products']) \
    .agg({'products': 'count'}) \
    .rename(columns={'products': 'purchase_count'}) \
    .reset_index() \
    .rename(columns={'products': 'productId'})

,customerId,productId,purchase_count
0,0,20.0,1
1,1,2.0,2
2,1,23.0,1
3,1,29.0,1
4,1,68.0,2
5,1,86.0,1
6,1,107.0,1
7,1,111.0,1
8,1,152.0,1


### 3.1. Create data with user, item, and target field
* This table will be an input for our modeling later
    * In this case, our user is `customerId`, `productId`, and `purchase_count`

In [0]:
s=time.time()

data = pd.melt(transactions.set_index('customerId')['products'].apply(pd.Series).reset_index(), 
             id_vars=['customerId'],
             value_name='products') \
    .dropna().drop(['variable'], axis=1) \
    .groupby(['customerId', 'products']) \
    .agg({'products': 'count'}) \
    .rename(columns={'products': 'purchase_count'}) \
    .reset_index() \
    .rename(columns={'products': 'productId'})
data['productId'] = data['productId'].astype(np.int64)

print("Execution time:", round((time.time()-s)/60,2), "minutes")

Execution time: 0.2 minutes


In [0]:
print(data.shape)
data.head()

(133585, 3)


,customerId,productId,purchase_count
0,0,1,2
1,0,13,1
2,0,19,3
3,0,20,1
4,0,31,2


### 3.2. Create dummy
* Dummy for marking whether a customer bought that item or not.
* If one buys an item, then `purchase_dummy` are marked as 1
* Why create a dummy instead of normalizing it, you ask?
    * Normalizing the purchase count, say by each user, would not work because customers may have different buying frequency don't have the same taste
    * However, we can normalize items by purchase frequency across all users, which is done in section 3.3. below.

In [0]:
def create_data_dummy(data):
    data_dummy = data.copy()
    data_dummy['purchase_dummy'] = 1
    return data_dummy

In [0]:
data_dummy = create_data_dummy(data)

### 3.3. Normalize item values across users
* To do this, we normalize purchase frequency of each item across users by first creating a user-item matrix as follows

In [0]:
df_matrix = pd.pivot_table(data, values='purchase_count', index='customerId', columns='productId')
df_matrix.head()

productId,0,1,2,3,4,5,6,7,8,9,...,290,291,292,293,294,295,296,297,298,299
customerId,,,,,,,,,,,,,,,,,,,,,
0,NaN,2.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,NaN,NaN,6.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,1.0,NaN,NaN,1.0,NaN,NaN,NaN
2,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [0]:
(df_matrix.shape)

(24429, 300)

In [0]:
df_matrix_norm = (df_matrix-df_matrix.min())/(df_matrix.max()-df_matrix.min())
print(df_matrix_norm.shape)
df_matrix_norm.head()

(24429, 300)


productId,0,1,2,3,4,5,6,7,8,9,...,290,291,292,293,294,295,296,297,298,299
customerId,,,,,,,,,,,,,,,,,,,,,
0,NaN,0.1,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,NaN,NaN,0.166667,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,0.0,NaN,NaN,0.0,NaN,NaN,NaN
2,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [0]:
# create a table for input to the modeling

d = df_matrix_norm.reset_index()
d.index.names = ['scaled_purchase_freq']
data_norm = pd.melt(d, id_vars=['customerId'], value_name='scaled_purchase_freq').dropna()
print(data_norm.shape)
data_norm.head()

(133585, 3)


,customerId,productId,scaled_purchase_freq
9,9,0,0.133333
25,25,0,0.133333
32,33,0,0.133333
35,36,0,0.133333
43,44,0,0.133333


#### Define a function for normalizing data

In [0]:
def normalize_data(data):
    df_matrix = pd.pivot_table(data, values='purchase_count', index='customerId', columns='productId')
    df_matrix_norm = (df_matrix-df_matrix.min())/(df_matrix.max()-df_matrix.min())
    d = df_matrix_norm.reset_index()
    d.index.names = ['scaled_purchase_freq']
    return pd.melt(d, id_vars=['customerId'], value_name='scaled_purchase_freq').dropna()

* We can normalize the their purchase history, from 0-1 (with 1 being the most number of purchase for an item and 0 being 0 purchase count for that item).

## 4. Split train and test set
* Splitting the data into training and testing sets is an important part of evaluating predictive modeling, in this case a collaborative filtering model. Typically, we use a larger portion of the data for training and a smaller portion for testing. 
* We use 80:20 ratio for our train-test set size.
* Our training portion will be used to develop a predictive model, while the other to evaluate the model's performance.
* Now that we have three datasets with purchase counts, purchase dummy, and scaled purchase counts, we would like to split each.

In [0]:
train, test = train_test_split(data, test_size = .2)
print(train.shape, test.shape)

(106868, 3) (26717, 3)


In [0]:
# Using turicreate library, we convert dataframe to SFrame - this will be useful in the modeling part

train_data = tc.SFrame(train)
test_data = tc.SFrame(test)

In [0]:
train_data

customerId,productId,purchase_count
6652,2,6
15801,221,1
7185,61,1
19649,49,2
11918,167,1
6005,3,3
15634,66,1
9065,247,1
2745,57,4
2302,72,2


In [0]:
test_data

customerId,productId,purchase_count
4210,69,1
502,95,2
10230,274,1
3756,21,1
14613,116,1
16462,182,1
148,288,1
5727,178,1
2073,76,2
16080,15,1


#### Define a `split_data` function for splitting data to training and test set

In [0]:
# We can define a function for this step as follows

def split_data(data):
    '''
    Splits dataset into training and test set.
    
    Args:
        data (pandas.DataFrame)
        
    Returns
        train_data (tc.SFrame)
        test_data (tc.SFrame)
    '''
    train, test = train_test_split(data, test_size = .2)
    train_data = tc.SFrame(train)
    test_data = tc.SFrame(test)
    return train_data, test_data

In [0]:
# lets try with both dummy table and scaled/normalized purchase table

train_data_dummy, test_data_dummy = split_data(data_dummy)
train_data_norm, test_data_norm = split_data(data_norm)

## 5. Baseline Model
Before running a more complicated approach such as collaborative filtering, we would like to use a baseline model to compare and evaluate models. Since baseline typically uses a very simple approach, techniques used beyond this approach should be chosen if they show relatively better accuracy and complexity.

### 5.1. Using a Popularity model as a baseline
* The popularity model takes the most popular items for recommendation. These items are products with the highest number of sells across customers.
* We use `turicreate` library for running and evaluating both baseline and collaborative filtering models below
* Training data is used for model selection

#### Using purchase counts

In [0]:
# variables to define field names
user_id = 'customerId'
item_id = 'productId'
target = 'purchase_count'
users_to_recommend = list(transactions[user_id])
n_rec = 10 # number of items to recommend
n_display = 30

In [0]:
popularity_model = tc.popularity_recommender.create(train_data, 
                                                    user_id=user_id, 
                                                    item_id=item_id, 
                                                    target=target)

Preparing data set.

Data has 106868 observations with 23360 users and 300 items.

Data prepared in: 0.156313s

106868 observations to process; with 300 unique items.

In [0]:
# Get recommendations for a list of users to recommend (from customers file)
# Printed below is head / top 30 rows for first 3 customers with 10 recommendations each

popularity_recomm = popularity_model.recommend(users=users_to_recommend, k=n_rec)
popularity_recomm.print_rows(n_display)

recommendations finished on 1000/62483 queries. users per second: 89952.3

recommendations finished on 2000/62483 queries. users per second: 88802.1

recommendations finished on 3000/62483 queries. users per second: 86727.8

recommendations finished on 4000/62483 queries. users per second: 87927.5

recommendations finished on 5000/62483 queries. users per second: 87716.2

...

recommendations finished on 59000/62483 queries. users per second: 84516

recommendations finished on 60000/62483 queries. users per second: 84394

recommendations finished on 61000/62483 queries. users per second: 84179.4

recommendations finished on 62000/62483 queries. users per second: 84101.3

+------------+-----------+--------------------+------+
| customerId | productId |       score        | rank |
+------------+-----------+--------------------+------+
|     0      |    132    | 3.1206896551724137 |  1   |
|     0      |    248    | 3.111111111111111  |  2   |
|     0      |     0     | 2.995164410058027  |  3   |
|     0      |     37    | 2.9808429118773945 |  4   |
|     0      |     34    | 2.973384030418251  |  5   |
|     0      |     27    | 2.8461538461538463 |  6   |
|     0      |     3     | 2.814814814814815  |  7   |
|     0      |    110    | 2.701863354037267  |  8   |
|     0      |     32    | 2.6778846153846154 |  9   |
|     0      |    230    | 2.6343283582089554 |  10  |
|     1      |    132    | 3.1206896551724137 |  1   |
|     1      |    248    | 3.111111111111111  |  2   |
|     1      |     0     | 2.995164410058027  |  3   |
|     1      |     37    | 2.9808429118773945 |  4   |
|     1      |     34    | 2.973384030418251  |  5   |
|     1   

#### Define a `model` function for model selection

In [0]:
# Since turicreate is very accessible library, we can define a model selection function as below

def model(train_data, name, user_id, item_id, target, users_to_recommend, n_rec, n_display):
    if name == 'popularity':
        model = tc.popularity_recommender.create(train_data, 
                                                    user_id=user_id, 
                                                    item_id=item_id, 
                                                    target=target)
    elif name == 'cosine':
        model = tc.item_similarity_recommender.create(train_data, 
                                                    user_id=user_id, 
                                                    item_id=item_id, 
                                                    target=target, 
                                                    similarity_type='cosine')
    elif name == 'pearson':
        model = tc.item_similarity_recommender.create(train_data, 
                                                    user_id=user_id, 
                                                    item_id=item_id, 
                                                    target=target, 
                                                    similarity_type='pearson')
        
    recom = model.recommend(users=users_to_recommend, k=n_rec)
    recom.print_rows(n_display)
    return model

In [0]:
# variables to define field names
# constant variables include:
user_id = 'customerId'
item_id = 'productId'
users_to_recommend = list(customers[user_id])
n_rec = 10 # number of items to recommend
n_display = 30 # to print the head / first few rows in a defined dataset

#### Using purchase dummy

In [0]:
# these variables will change accordingly
name = 'popularity'
target = 'purchase_dummy'
pop_dummy = model(train_data_dummy, name, user_id, item_id, target, users_to_recommend, n_rec, n_display)

Warning: Ignoring columns purchase_count;

To use these columns in scoring predictions, use a model that allows the use of additional features.

Preparing data set.

Data has 106868 observations with 23300 users and 300 items.

Data prepared in: 0.155083s

106868 observations to process; with 300 unique items.

recommendations finished on 1000/1000 queries. users per second: 92618.3

+------------+-----------+-------+------+
| customerId | productId | score | rank |
+------------+-----------+-------+------+
|    1553    |     57    |  1.0  |  1   |
|    1553    |     63    |  1.0  |  2   |
|    1553    |    299    |  1.0  |  3   |
|    1553    |    203    |  1.0  |  4   |
|    1553    |     48    |  1.0  |  5   |
|    1553    |    206    |  1.0  |  6   |
|    1553    |     71    |  1.0  |  7   |
|    1553    |    209    |  1.0  |  8   |
|    1553    |    260    |  1.0  |  9   |
|    1553    |     26    |  1.0  |  10  |
|   20400    |     57    |  1.0  |  1   |
|   20400    |     63    |  1.0  |  2   |
|   20400    |    299    |  1.0  |  3   |
|   20400    |    203    |  1.0  |  4   |
|   20400    |     48    |  1.0  |  5   |
|   20400    |    206    |  1.0  |  6   |
|   20400    |     71    |  1.0  |  7   |
|   20400    |    209    |  1.0  |  8   |
|   20400    |    260    |  1.0  |  9   |
|   20400    |     26    |  1.0  |  10  |
|   19750    |     57    |  1.0  |

#### Using normalized purchase count

In [0]:
name = 'popularity'
target = 'scaled_purchase_freq'
pop_norm = model(train_data_norm, name, user_id, item_id, target, users_to_recommend, n_rec, n_display)

Preparing data set.

Data has 106868 observations with 23251 users and 300 items.

Data prepared in: 0.16011s

106868 observations to process; with 300 unique items.

recommendations finished on 1000/1000 queries. users per second: 80723.3

+------------+-----------+---------------------+------+
| customerId | productId |        score        | rank |
+------------+-----------+---------------------+------+
|    1553    |    226    |  0.7647058823529411 |  1   |
|    1553    |    247    |  0.3387096774193547 |  2   |
|    1553    |    230    |  0.3230769230769226 |  3   |
|    1553    |    294    | 0.26315789473684187 |  4   |
|    1553    |    125    |  0.2558139534883717 |  5   |
|    1553    |    204    | 0.25054945054945027 |  6   |
|    1553    |    248    |       0.234375      |  7   |
|    1553    |    276    | 0.23295454545454544 |  8   |
|    1553    |    155    | 0.23272727272727264 |  9   |
|    1553    |     72    |  0.2310838445807771 |  10  |
|   20400    |    226    |  0.7647058823529411 |  1   |
|   20400    |    247    |  0.3387096774193547 |  2   |
|   20400    |    230    |  0.3230769230769226 |  3   |
|   20400    |    294    | 0.26315789473684187 |  4   |
|   20400    |    125    |  0.2558139534883717 |

#### Notes
* Once we created the model, we predicted the recommendation items using scores by popularity. As you can tell for each model results above, the rows show the first 30 records from 1000 users with 10 recommendations. These 30 records include 3 users and their recommended items, along with score and descending ranks. 
* In the result, although different models have different recommendation list, each user is recommended the same list of 10 items. This is because popularity is calculated by taking the most popular items across all users.
* If a grouping example below, products 132, 248, 37, and 34 are the most popular (best-selling) across customers. Using their purchase counts divided by the number of customers, we see that these products are at least bought 3 times on average in the training set of transactions (same as the first popularity measure on `purchase_count` variable)

In [0]:
train.groupby(by=item_id)['purchase_count'].mean().sort_values(ascending=False).head(20)

productId
132    3.120690
248    3.111111
0      2.995164
37     2.980843
34     2.973384
27     2.846154
3      2.814815
110    2.701863
32     2.677885
230    2.634328
10     2.610119
82     2.579167
226    2.539474
245    2.534091
83     2.444444
91     2.442623
41     2.432099
58     2.428571
129    2.427746
87     2.399177
Name: purchase_count, dtype: float64

## 6. Collaborative Filtering Model

* In collaborative filtering, we would recommend items based on how similar users purchase items. For instance, if customer 1 and customer 2 bought similar items, e.g. 1 bought X, Y, Z and 2 bought X, Y, we would recommend an item Z to customer 2.

* To define similarity across users, we use the following steps:
    1. Create a user-item matrix, where index values represent unique customer IDs and column values represent unique product IDs
    
    2. Create an item-to-item similarity matrix. The idea is to calculate how similar a product is to another product. There are a number of ways of calculating this. In steps 6.1 and 6.2, we use cosine and pearson similarity measure, respectively.  
    
        * To calculate similarity between products X and Y, look at all customers who have rated both these items. For example, both X and Y have been rated by customers 1 and 2. 
        * We then create two item-vectors, v1 for item X and v2 for item Y, in the user-space of (1, 2) and then find the `cosine` or `pearson` angle/distance between these vectors. A zero angle or overlapping vectors with cosine value of 1 means total similarity (or per user, across all items, there is same rating) and an angle of 90 degree would mean cosine of 0 or no similarity.
        
    3. For each customer, we then predict his likelihood to buy a product (or his purchase counts) for products that he had not bought. 
    
        * For our example, we will calculate rating for user 2 in the case of item Z (target item). To calculate this we weigh the just-calculated similarity-measure between the target item and other items that customer has already bought. The weighing factor is the purchase counts given by the user to items already bought by him. 
        * We then scale this weighted sum with the sum of similarity-measures so that the calculated rating remains within a predefined limits. Thus, the predicted rating for item Z for user 2 would be calculated using similarity measures.

* While I wrote python scripts for all the process including finding similarity using python scripts (which can be found in `scripts` folder, we can use `turicreate` library for now to capture different measures like using `cosine` and `pearson` distance, and evaluate the best model.

### 6.1. `Cosine` similarity
* Similarity is the cosine of the angle between the 2 vectors of the item vectors of A and B
* It is defined by the following formula
![](https://encrypted-tbn0.gstatic.com/images?q=tbn:ANd9GcTnRHSAx1c084UXF2wIHYwaHJLmq2qKtNk_YIv3RjHUO00xwlkt)
* Closer the vectors, smaller will be the angle and larger the cosine

#### Using purchase count

In [0]:
# these variables will change accordingly
name = 'cosine'
target = 'purchase_count'
cos = model(train_data, name, user_id, item_id, target, users_to_recommend, n_rec, n_display)

Preparing data set.

Data has 106868 observations with 23360 users and 300 items.

Data prepared in: 0.150669s

Training model from provided data.

Gathering per-item and per-user statistics.

...

Finalizing lookup tables.

Generating candidate set for working with new users.

Finished training in 1.0688s

recommendations finished on 1000/1000 queries. users per second: 50854.4

+------------+-----------+----------------------+------+
| customerId | productId |        score         | rank |
+------------+-----------+----------------------+------+
|    1553    |     2     | 0.11276702880859375  |  1   |
|    1553    |     35    | 0.08033981323242187  |  2   |
|    1553    |     1     | 0.07192964553833008  |  3   |
|    1553    |     5     |  0.0660744309425354  |  4   |
|    1553    |     21    | 0.05912514925003052  |  5   |
|    1553    |     8     | 0.058310282230377194 |  6   |
|    1553    |     17    | 0.05692689418792725  |  7   |
|    1553    |     33    | 0.04889466762542725  |  8   |
|    1553    |     15    | 0.04863626956939697  |  9   |
|    1553    |     41    | 0.048443269729614255 |  10  |
|   20400    |    182    | 0.03934735059738159  |  1   |
|   20400    |     9     | 0.039135098457336426 |  2   |
|   20400    |     75    | 0.03855240345001221  |  3   |
|   20400    |    122    | 0.03753024339675903  |  4   |
|   20400    |     6     | 0.03

#### Using purchase dummy

In [0]:
# these variables will change accordingly
name = 'cosine'
target = 'purchase_dummy'
cos_dummy = model(train_data_dummy, name, user_id, item_id, target, users_to_recommend, n_rec, n_display)

Warning: Ignoring columns purchase_count;

To use these columns in scoring predictions, use a model that allows the use of additional features.

Preparing data set.

Data has 106868 observations with 23300 users and 300 items.

Data prepared in: 0.145507s

...

Finalizing lookup tables.

Generating candidate set for working with new users.

Finished training in 0.062704s

recommendations finished on 1000/1000 queries. users per second: 28282.9

+------------+-----------+----------------------+------+
| customerId | productId |        score         | rank |
+------------+-----------+----------------------+------+
|    1553    |     2     |  0.1002994179725647  |  1   |
|    1553    |     35    | 0.08169955015182495  |  2   |
|    1553    |     1     | 0.07999366521835327  |  3   |
|    1553    |     5     | 0.07133170366287231  |  4   |
|    1553    |     21    | 0.06368176937103272  |  5   |
|    1553    |     17    |  0.0635490894317627  |  6   |
|    1553    |     61    |  0.0536016583442688  |  7   |
|    1553    |     33    | 0.050506067276000974 |  8   |
|    1553    |     8     | 0.049720931053161624 |  9   |
|    1553    |     96    | 0.045212066173553465 |  10  |
|   20400    |     57    |         0.0          |  1   |
|   20400    |     63    |         0.0          |  2   |
|   20400    |    299    |         0.0          |  3   |
|   20400    |    203    |         0.0          |  4   |
|   20400    |     48    |     

#### Using normalized purchase count

In [0]:
name = 'cosine'
target = 'scaled_purchase_freq'
cos_norm = model(train_data_norm, name, user_id, item_id, target, users_to_recommend, n_rec, n_display)

Preparing data set.

Data has 106868 observations with 23251 users and 300 items.

Data prepared in: 0.162553s

Training model from provided data.

Gathering per-item and per-user statistics.

...

Finalizing lookup tables.

Generating candidate set for working with new users.

Finished training in 0.069365s

recommendations finished on 1000/1000 queries. users per second: 40394.2

+------------+-----------+-----------------------+------+
| customerId | productId |         score         | rank |
+------------+-----------+-----------------------+------+
|    1553    |     53    |          0.0          |  1   |
|    1553    |     63    |          0.0          |  2   |
|    1553    |    202    |          0.0          |  3   |
|    1553    |    297    |          0.0          |  4   |
|    1553    |    153    |          0.0          |  5   |
|    1553    |    258    |          0.0          |  6   |
|    1553    |     7     |          0.0          |  7   |
|    1553    |    108    |          0.0          |  8   |
|    1553    |     97    |          0.0          |  9   |
|    1553    |    210    |          0.0          |  10  |
|   20400    |     1     |  0.003668515682220459 |  1   |
|   20400    |     2     |  0.003234623670578003 |  2   |
|   20400    |     8     | 0.0027580273151397703 |  3   |
|   20400    |     0     |  0.002273153066635132 |  4   |
|   20400    |

### 6.2. `Pearson` similarity
* Similarity is the pearson coefficient between the two vectors.
* It is defined by the following formula
![](http://critical-numbers.group.shef.ac.uk/glossary/images/correlationKT1.png)

#### Using purchase count

In [0]:
# these variables will change accordingly
name = 'pearson'
target = 'purchase_count'
pear = model(train_data, name, user_id, item_id, target, users_to_recommend, n_rec, n_display)

Preparing data set.

Data has 106868 observations with 23360 users and 300 items.

Data prepared in: 0.149624s

Training model from provided data.

Gathering per-item and per-user statistics.

...

Finalizing lookup tables.

Generating candidate set for working with new users.

Finished training in 0.106142s

recommendations finished on 1000/1000 queries. users per second: 62270.4

+------------+-----------+--------------------+------+
| customerId | productId |       score        | rank |
+------------+-----------+--------------------+------+
|    1553    |    132    | 3.1206896551724137 |  1   |
|    1553    |    248    | 3.1084946102566193 |  2   |
|    1553    |     0     | 2.9932216205034314 |  3   |
|    1553    |     37    | 2.979803561844588  |  4   |
|    1553    |     34    | 2.9559458213613987 |  5   |
|    1553    |     27    | 2.846153846153846  |  6   |
|    1553    |     3     | 2.8143691151230428 |  7   |
|    1553    |    110    | 2.6935129340390978 |  8   |
|    1553    |     32    | 2.6778846153846163 |  9   |
|    1553    |    230    | 2.6287490058301097 |  10  |
|   20400    |    248    | 3.1111111111111116 |  1   |
|   20400    |    132    | 3.1104909428234757 |  2   |
|   20400    |     0     | 2.9928007878694594 |  3   |
|   20400    |     37    | 2.9808429118773945 |  4   |
|   20400    |     34    | 2.973384030418253  |  5   |
|   20400 

#### Using purchase dummy

In [0]:
# these variables will change accordingly
name = 'pearson'
target = 'purchase_dummy'
pear_dummy = model(train_data_dummy, name, user_id, item_id, target, users_to_recommend, n_rec, n_display)

Warning: Ignoring columns purchase_count;

To use these columns in scoring predictions, use a model that allows the use of additional features.

Preparing data set.

Data has 106868 observations with 23300 users and 300 items.

Data prepared in: 0.14787s

...

Finalizing lookup tables.

Generating candidate set for working with new users.

Finished training in 0.082016s

recommendations finished on 1000/1000 queries. users per second: 53954.9

+------------+-----------+-------+------+
| customerId | productId | score | rank |
+------------+-----------+-------+------+
|    1553    |     57    |  0.0  |  1   |
|    1553    |     63    |  0.0  |  2   |
|    1553    |    299    |  0.0  |  3   |
|    1553    |    203    |  0.0  |  4   |
|    1553    |     48    |  0.0  |  5   |
|    1553    |    206    |  0.0  |  6   |
|    1553    |     71    |  0.0  |  7   |
|    1553    |    209    |  0.0  |  8   |
|    1553    |    260    |  0.0  |  9   |
|    1553    |     26    |  0.0  |  10  |
|   20400    |     57    |  0.0  |  1   |
|   20400    |     63    |  0.0  |  2   |
|   20400    |    299    |  0.0  |  3   |
|   20400    |    203    |  0.0  |  4   |
|   20400    |     48    |  0.0  |  5   |
|   20400    |    206    |  0.0  |  6   |
|   20400    |     71    |  0.0  |  7   |
|   20400    |    209    |  0.0  |  8   |
|   20400    |    260    |  0.0  |  9   |
|   20400    |     26    |  0.0  |  10  |
|   19750    |     57    |  0.0  |

#### Using normalized purchase count

In [0]:
name = 'pearson'
target = 'scaled_purchase_freq'
pear_norm = model(train_data_norm, name, user_id, item_id, target, users_to_recommend, n_rec, n_display)

Preparing data set.

Data has 106868 observations with 23251 users and 300 items.

Data prepared in: 0.153997s

Training model from provided data.

Gathering per-item and per-user statistics.

...

Finalizing lookup tables.

Generating candidate set for working with new users.

Finished training in 0.102272s

recommendations finished on 1000/1000 queries. users per second: 59673

+------------+-----------+---------------------+------+
| customerId | productId |        score        | rank |
+------------+-----------+---------------------+------+
|    1553    |    226    |  0.7645709612790277 |  1   |
|    1553    |    247    |  0.3387096774193547 |  2   |
|    1553    |    230    |  0.3230769230769232 |  3   |
|    1553    |    294    | 0.26315789473684204 |  4   |
|    1553    |    125    |  0.255813953488372  |  5   |
|    1553    |    204    | 0.25054945054945055 |  6   |
|    1553    |    248    | 0.23385876417160034 |  7   |
|    1553    |    276    |  0.2327507929368452 |  8   |
|    1553    |    155    |  0.2327272727272727 |  9   |
|    1553    |     72    |  0.231083844580777  |  10  |
|   20400    |    226    |   0.76469501284992  |  1   |
|   20400    |    247    | 0.33870578881232954 |  2   |
|   20400    |    230    | 0.32306969303351196 |  3   |
|   20400    |    294    |  0.2631339610877789 |  4   |
|   20400    |    125    | 0.25580356201460186 |

#### Note
* In collaborative filtering above, we used two approaches: cosine and pearson distance. We also got to apply them to three training datasets with normal counts, dummy, or normalized counts of items purchase.
* We can see that the recommendations are different for each user. This suggests that personalization does exist. 
* But how good is this model compared to the baseline, and to each other? We need some means of evaluating a recommendation engine. Lets focus on that in the next section.

## 7. Model Evaluation
For evaluating recommendation engines, we can use the concept of precision-recall.

* RMSE (Root Mean Squared Errors)
    * Measures the error of predicted values
    * Lesser the RMSE value, better the recommendations
* Recall
    * What percentage of products that a user buys are actually recommended?
    * If a customer buys 5 products and the recommendation decided to show 3 of them, then the recall is 0.6
* Precision
    * Out of all the recommended items, how many the user actually liked?
    * If 5 products were recommended to the customer out of which he buys 4 of them, then precision is 0.8
    
* Why are both recall and precision important?
    * Consider a case where we recommend all products, so our customers will surely cover the items that they liked and bought. In this case, we have 100% recall! Does this mean our model is good?
    * We have to consider precision. If we recommend 300 items but user likes and buys only 3 of them, then precision is 0.1%! This very low precision indicates that the model is not great, despite their excellent recall.
    * So our aim has to be optimizing both recall and precision (to be close to 1 as possible).

Lets compare all the models we have built based on precision-recall characteristics:

In [0]:
# create initial callable variables

models_w_counts = [popularity_model, cos, pear]
models_w_dummy = [pop_dummy, cos_dummy, pear_dummy]
models_w_norm = [pop_norm, cos_norm, pear_norm]

names_w_counts = ['Popularity Model on Purchase Counts', 'Cosine Similarity on Purchase Counts', 'Pearson Similarity on Purchase Counts']
names_w_dummy = ['Popularity Model on Purchase Dummy', 'Cosine Similarity on Purchase Dummy', 'Pearson Similarity on Purchase Dummy']
names_w_norm = ['Popularity Model on Scaled Purchase Counts', 'Cosine Similarity on Scaled Purchase Counts', 'Pearson Similarity on Scaled Purchase Counts']

#### Models on purchase counts

In [0]:
eval_counts = tc.recommender.util.compare_models(test_data, models_w_counts, model_names=names_w_counts)

PROGRESS: Evaluate model Popularity Model on Purchase Counts


recommendations finished on 1000/13858 queries. users per second: 48906.9

recommendations finished on 2000/13858 queries. users per second: 45413.3

recommendations finished on 3000/13858 queries. users per second: 47023.4

recommendations finished on 4000/13858 queries. users per second: 47789.2

...

recommendations finished on 10000/13858 queries. users per second: 35552.6

recommendations finished on 11000/13858 queries. users per second: 35648.2

recommendations finished on 12000/13858 queries. users per second: 35225.6

recommendations finished on 13000/13858 queries. users per second: 35460.8


Precision and recall summary statistics by cutoff
+--------+-----------------------+----------------------+
| cutoff |     mean_precision    |     mean_recall      |
+--------+-----------------------+----------------------+
|   1    | 0.0010824072737768778 | 0.000650647039014767 |
|   2    | 0.0009020060614807327 | 0.000978375908019435 |
|   3    |  0.007312262471737147 | 0.01166531108441918  |
|   4    |  0.006728965218646288 | 0.014321160550775166 |
|   5    |  0.006032616539183138 | 0.01611454904679024  |
|   6    |  0.00549622360128926  | 0.017702537876805424 |
|   7    |  0.005948085685421523 | 0.021931085423292078 |
|   8    |  0.005682638187328629 | 0.023829503108619775 |
|   9    |  0.005412036368884418 | 0.026254465455648787 |
|   10   |  0.005181122817145299 | 0.028121618002913875 |
+--------+-----------------------+----------------------+
[10 rows x 3 columns]


Overall RMSE: 1.0553182318397212

Per User RMSE (best)
+------------+-----------------------+-------+
| customerI

#### Models on purchase dummy

In [0]:
eval_dummy = tc.recommender.util.compare_models(test_data_dummy, models_w_dummy, model_names=names_w_dummy)

PROGRESS: Evaluate model Popularity Model on Purchase Dummy


recommendations finished on 1000/13982 queries. users per second: 51216.4

recommendations finished on 2000/13982 queries. users per second: 47760.1

recommendations finished on 3000/13982 queries. users per second: 49386

recommendations finished on 4000/13982 queries. users per second: 49975

...

recommendations finished on 10000/13982 queries. users per second: 44085.5

recommendations finished on 11000/13982 queries. users per second: 44726.5

recommendations finished on 12000/13982 queries. users per second: 45339.1

recommendations finished on 13000/13982 queries. users per second: 43162.3


Precision and recall summary statistics by cutoff
+--------+----------------------+-----------------------+
| cutoff |    mean_precision    |      mean_recall      |
+--------+----------------------+-----------------------+
|   1    | 0.006508367901587766 | 0.0031807644667035217 |
|   2    | 0.006079244743241322 |  0.006144891163128905 |
|   3    | 0.005912363514995462 |  0.009036591319168901 |
|   4    | 0.005828922900872541 |  0.011584853225891662 |
|   5    | 0.005549992847947367 |  0.013664612383850767 |
|   6    | 0.005483240356649042 |  0.01653340417947707  |
|   7    | 0.00552751496822447  |  0.01980080329520372  |
|   8    | 0.005507080532112723 |  0.02266875470024475  |
|   9    | 0.005451453456030793 |  0.02532336531298653  |
|   10   | 0.005399799742526085 |  0.02794777129528115  |
+--------+----------------------+-----------------------+
[10 rows x 3 columns]


Overall RMSE: 1.0

Per User RMSE (best)
+------------+------+-------+
| customerId | rmse | count |
+------------

#### Models on normalized purchase frequency

In [0]:
eval_norm = tc.recommender.util.compare_models(test_data_norm, models_w_norm, model_names=names_w_norm)

PROGRESS: Evaluate model Popularity Model on Scaled Purchase Counts


recommendations finished on 1000/13967 queries. users per second: 46347.8

recommendations finished on 2000/13967 queries. users per second: 43673.8

recommendations finished on 3000/13967 queries. users per second: 44554.7

recommendations finished on 4000/13967 queries. users per second: 44556.8

...

recommendations finished on 10000/13967 queries. users per second: 33893.9

recommendations finished on 11000/13967 queries. users per second: 34161.3

recommendations finished on 12000/13967 queries. users per second: 33837.9

recommendations finished on 13000/13967 queries. users per second: 34093.4


Precision and recall summary statistics by cutoff
+--------+-----------------------+-----------------------+
| cutoff |     mean_precision    |      mean_recall      |
+--------+-----------------------+-----------------------+
|   1    |  0.001933128087635137 | 0.0010119942585754868 |
|   2    |  0.002219517433951469 | 0.0019076952974712033 |
|   3    | 0.0023627121071096145 |  0.003805024716816791 |
|   4    | 0.0023269134388200674 |  0.005169635858044252 |
|   5    | 0.0024343094436886834 |  0.006838649326298788 |
|   6    | 0.0022553161022409907 |   0.0077002039431337  |
|   7    |  0.002096779142673026 |  0.00834580425580641  |
|   8    | 0.0020852724278656947 |  0.009307794025342516 |
|   9    | 0.0021001885396529866 |  0.010349918829728374 |
|   10   |  0.002670580654399656 |  0.01464141462620021  |
+--------+-----------------------+-----------------------+
[10 rows x 3 columns]


Overall RMSE: 0.13192690523465408

Per User RMSE (best)
+------------+------------------------+----

## 8. Model Selection
### 8.1. Evaluation summary
* Based on RMSE


    1. Popularity on purchase counts: 1.1111750034210488
    2. Cosine similarity on purchase counts: 1.9230643981653215
    3. Pearson similarity on purchase counts: 1.9231102838192284
    
    4. Popularity on purchase dummy: 0.9697374361161925
    5. Cosine similarity on purchase dummy: 0.9697509978436404
    6. Pearson similarity on purchase dummy: 0.9697745320187097
    
    7. Popularity on scaled purchase counts: 0.16230660626840343
    8. Cosine similarity on scaled purchase counts: 0.16229800354111104
    9. Pearson similarity on scaled purchase counts: 0.1622982668334026
    
* Based on Precision and Recall
![](../images/model_comparisons.png)


#### Notes

* Popularity v. Collaborative Filtering: We can see that the collaborative filtering algorithms work better than popularity model for purchase counts. Indeed, popularity model doesn’t give any personalizations as it only gives the same list of recommended items to every user.
* Precision and recall: Looking at the summary above, we see that the precision and recall for Purchase Counts > Purchase Dummy > Normalized Purchase Counts. However, because the recommendation scores for the normalized purchase data is zero and constant, we choose the dummy. In fact, the RMSE isn’t much different between models on the dummy and those on the normalized data.
* RMSE: Since RMSE is higher using pearson distance thancosine, we would choose model the smaller mean squared errors, which in this case would be cosine.
Therefore, we select the Cosine similarity on Purchase Dummy approach as our final model.

## 8. Final Output
* In this step, we would like to manipulate format for recommendation output to one we can export to csv, and also a function that will return recommendation list given a customer ID.
* We need to first rerun the model using the whole dataset, as we came to a final model using train data and evaluated with test set.

In [0]:
users_to_recommend = list(customers[user_id])

final_model = tc.item_similarity_recommender.create(tc.SFrame(data_dummy), 
                                            user_id=user_id, 
                                            item_id=item_id, 
                                            target='purchase_dummy', 
                                            similarity_type='cosine')

recom = final_model.recommend(users=users_to_recommend, k=n_rec)
recom.print_rows(n_display)

Warning: Ignoring columns purchase_count;

To use these columns in scoring predictions, use a model that allows the use of additional features.

Preparing data set.

Data has 133585 observations with 24429 users and 300 items.

Data prepared in: 0.148222s

...

Finalizing lookup tables.

Generating candidate set for working with new users.

Finished training in 0.083134s

recommendations finished on 1000/1000 queries. users per second: 39801

+------------+-----------+----------------------+------+
| customerId | productId |        score         | rank |
+------------+-----------+----------------------+------+
|    1553    |     2     | 0.11079320907592774  |  1   |
|    1553    |     35    | 0.10447167158126831  |  2   |
|    1553    |     1     | 0.10348175764083863  |  3   |
|    1553    |     5     | 0.07978436946868897  |  4   |
|    1553    |     17    | 0.06781145334243774  |  5   |
|    1553    |     33    |  0.0668614387512207  |  6   |
|    1553    |     61    | 0.06512556076049805  |  7   |
|    1553    |     15    | 0.06476415395736694  |  8   |
|    1553    |     21    |  0.063294517993927   |  9   |
|    1553    |     11    | 0.05467898845672607  |  10  |
|   20400    |     26    | 0.05812269449234009  |  1   |
|   20400    |     6     | 0.05361741781234741  |  2   |
|   20400    |    113    | 0.05312788486480713  |  3   |
|   20400    |     1     | 0.05210459232330322  |  4   |
|   20400    |     15    | 0.04

### 8.1. CSV output file

In [0]:
df_rec = recom.to_dataframe()
print(df_rec.shape)
df_rec.head()

(10000, 4)


,customerId,productId,score,rank
0,1553,2,0.110793,1
1,1553,35,0.104472,2
2,1553,1,0.103482,3
3,1553,5,0.079784,4
4,1553,17,0.067811,5


In [0]:
df_rec['recommendedProducts'] = df_rec.groupby([user_id])[item_id].transform(lambda x: '|'.join(x.astype(str)))
df_output = df_rec[['customerId', 'recommendedProducts']].drop_duplicates().sort_values('customerId').set_index('customerId')

#### Define a function to create a desired output

In [0]:
def create_output(model, users_to_recommend, n_rec, print_csv=True):
    recomendation = model.recommend(users=users_to_recommend, k=n_rec)
    df_rec = recomendation.to_dataframe()
    df_rec['recommendedProducts'] = df_rec.groupby([user_id])[item_id] \
        .transform(lambda x: '|'.join(x.astype(str)))
    df_output = df_rec[['customerId', 'recommendedProducts']].drop_duplicates() \
        .sort_values('customerId').set_index('customerId')
    if print_csv:
        df_output.to_csv('option1_recommendation.csv')
        print("An output file can be found in 'output' folder with name 'option1_recommendation.csv'")
    return df_output

In [0]:
df_output = create_output(pear_norm, users_to_recommend, n_rec, print_csv=True)
print(df_output.shape)
df_output.head()

recommendations finished on 1000/1000 queries. users per second: 51332.1

An output file can be found in 'output' folder with name 'option1_recommendation.csv'
(1000, 1)


,recommendedProducts
customerId,
4,226|247|230|294|125|204|248|276|155|72
11,226|247|230|294|125|204|248|276|155|83
12,226|247|230|294|125|204|248|276|155|72
16,226|247|230|294|125|204|248|276|155|72
21,226|247|230|294|125|204|248|276|155|72


### 8.2. Customer recommendation function

In [0]:
def customer_recomendation(customer_id):
    if customer_id not in df_output.index:
        print('Customer not found.')
        return customer_id
    return df_output.loc[customer_id]

In [0]:
customer_recomendation(4)

recommendedProducts    226|247|230|294|125|204|248|276|155|72
Name: 4, dtype: object

In [0]:
customer_recomendation(21)

recommendedProducts    226|247|230|294|125|204|248|276|155|72
Name: 21, dtype: object

## Summary
In this exercise, we were able to traverse a step-by-step process for making recommendations to customers. We used Collaborative Filtering approaches with `cosine` and `pearson` measure and compare the models with our baseline popularity model. We also prepared three sets of data that include regular buying count, buying dummy, as well as normalized purchase frequency as our target variable. Using RMSE, precision and recall, we evaluated our models and observed the impact of personalization. Finally, we selected the Cosine approach in dummy purchase data. 